# From Keras to ONNX: A Complete Guide to Jacobinet Backward Model Serialization

In this tutorial, we demonstrate how to serialize a Jacobinet model using Keras native serialization tools. We also show how to export Jacobinet models into the ONNX format, which is essential for industrial end-to-end pipelines.

**Why this matters**:
- **Portability**: ONNX models can be used across different platforms and frameworks.
- **Interoperability**: The serialized Keras model can be reloaded and validated, ensuring no corruption occurs.


- When running this notebook on Colab, we need to install *decomon* if on Colab. 
- If you run this notebook locally, do it inside the environment in which you [installed *jacobinet*](https://ducoffeM.github.io/jacobinet/main/install.html).

In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import sys  # noqa: avoid having this import removed by pycln

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install git+https://github.com/ducoffeM/jacobinet@main#egg=decomon
    # install desired backend (by default torch)
    !{sys.executable} -m pip install "torch"
    !{sys.executable} -m pip install "keras"

    # extra librabry used in this notebook
    !{sys.executable} -m pip install "onnx"
    !{sys.executable} -m pip install "onnxruntime"

In [ ]:
# Set this environment variable *before* importing torch, otherwise it has no effect.
# Ideally, we'd only set this if torch.backends.mps.is_available() is True,
# but checking that requires importing torch first, which would make this setting too late.
# So we preemptively enable the MPS fallback just in case MPS is available.
import os

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

## Step 1: Define the Forward Model

We will create a simple feedforward neural network with the following architecture:
- **Input**: Shape `(3,)`
- **Dense Layer**: 10 units + ReLU activation
- **Dense Layer**: 1 unit (output layer)

This model will be used as the basis for computing the Jacobian later on.

In [ ]:
# Import necessary libraries
import keras
from keras.layers import Activation, Dense, Input
from keras.models import Sequential

# Build the forward model
model = Sequential(
    [
        Dense(10, input_shape=(3,), name="Dense1"),
        Activation("relu", name="ReLU1"),
        Dense(1, name="Output"),
    ]
)

# Generate a forward pass to initialize model weights
_ = model(Input((3,)))

# Display model summary
print("### Forward Model Summary")
model.summary()

## Step 2: Compute the Backward Model Using Jacobinet

Jacobinet allows us to compute a **backward model** that represents the gradient of the output with respect to the input. This is key for understanding the chain rule in neural networks, fundamental to backpropagation.

We will now compute the backward model using the `clone_to_backward` function from Jacobinet.

In [ ]:
# Import Jacobinet and convert the forward model to a backward model
import jacobinet
from jacobinet import clone_to_backward

backward_model = clone_to_backward(model)

We can save this backward model using Keras' native saving and loading functions.

In [ ]:
# Save and load the backward model using Keras serialization
import keras

keras.models.save_model(backward_model, "my_backward_model.keras")

# Load the backward model
my_loaded_backward_model = keras.models.load_model("my_backward_model.keras")

# Generate random test input and gradient
import numpy as np

random_input = np.random.rand(3)[None].astype("float32")  # Batch size 1, 3 features
random_grad = np.ones((1, 1), dtype="float32")

# Check for model consistency (ensure loaded model matches original model)
assert np.allclose(
    backward_model.predict([random_input, random_grad]),
    my_loaded_backward_model.predict([random_input, random_grad]),
), "Loaded backward model does not match original!"

keras_jacobian = backward_model.predict([random_input, random_grad])

## Step 3: Export Backward Model to ONNX

**Why ONNX?**
- ONNX allows the model to be run on different platforms and inference engines.
- This enables industrial-level interoperability and simplifies deployment.

**Requirements**: 
- `torch` and `onnx` libraries are required for exporting the model.


In [ ]:
import torch
import torch.nn as nn

# Define a Torch wrapper around the Keras model


class Keras2Torch(nn.Module):
    def __init__(self, keras_model):
        super().__init__()
        self.keras_model = keras_model

    def forward(self, x, y):
        z = self.keras_model([x, y])
        return z


# Wrap the Keras backward model in the PyTorch interface
torch_model = Keras2Torch(backward_model)

# Generate random input and gradient for Torch
torch_input = torch.randn(1, 3)  # Batch size 1, 3 features
torch_grad = torch.ones(1, 1)  # Gradient size matches the output

# Run a forward pass to ensure no errors
_ = torch_model(torch_input, torch_grad)

# Export the backward model to ONNX format
torch.onnx.export(
    torch_model,  # Model to export
    (torch_input, torch_grad),  # Model inputs (as a tuple)
    "backward_model_torch.onnx",  # File name to save as
    input_names=["input_x", "input_grad"],
    output_names=["output"],
    dynamic_axes={
        "input_x": {0: "batch_size"},
        "output": {0: "batch_size"},
    },  # Handle batch size changes
)

print("ONNX model successfully exported as 'backward_model_torch.onnx'")

## Step 4: Validate the ONNX Model

We can use `onnx` to load and check the model, ensuring there is no corruption.


In [ ]:
import onnx

# Load the exported ONNX model
onnx_model = onnx.load("backward_model_torch.onnx")

# Check the model for errors
try:
    onnx.checker.check_model(onnx_model)
    print("ONNX model is valid!")
except Exception as e:
    print("ONNX model validation failed:", e)

## Step 5: Inference Using ONNX Runtime

To ensure the exported model works as expected, we'll run inference on the ONNX model using **ONNX Runtime**.

In [ ]:
import onnxruntime as ort

# Create an ONNX runtime session
ort_sess = ort.InferenceSession("backward_model_torch.onnx")

# Print the input and output names of the ONNX graph
print("Input names:", [input.name for input in ort_sess.get_inputs()])
print("Output names:", [output.name for output in ort_sess.get_outputs()])

# Run inference using the same random inputs
onnx_inputs = {"input_x": random_input, "input_grad": random_grad}
onnx_jacobian = ort_sess.run(None, onnx_inputs)

# Compare ONNX inference with the Keras jacobian
assert np.allclose(
    onnx_jacobian[0], keras_jacobian, atol=1e-5
), "ONNX inference does not match Keras!"

print(
    f"ONNX Inference Successful! Predicted: {onnx_jacobian[0]}, Keras Prediction: {keras_jacobian}"
)

## Step 6: Visualize the ONNX Model

You can visualize the structure of the ONNX model using **Netron**, which makes it easy to debug and understand model structure.

**Installation**:
```bash
pip install netron
```

**Visualization**:
```bash
netron -b backward_model_torch.onnx
```

Netron will open a web page showing the full model graph, its layers, and data flow.

---

With this process, you have a full industrial-grade pipeline for converting Jacobinet models to ONNX. This allows you to use your model in various environments, ensuring high standards for ARP compliance and reproducibility.
